In [1]:
import random
import numpy as np
import pandas as pd

# Optimizing Stardew Valley Farming
## Background
When I am not busy with schoolwork, I like to play a game called Stardew Valley. The premise of the game is that your character inherits a farm from your late grandfather and use the land to grow crops in order to earn money. While there are other features, such as fishing, slaying monsters, and making friends with the villagers, optimising the farming component of this game is what will be focused on. This is because crops are usually the most lucrative part of the game while also requiring the most foresight, since all crops take multiple in-game days to grow.

The crops available during the time period in game are as follows. All prices are in gold (the in-game currency)<sup>[1]</sup>:

| Crop        | Seed Price (Pierre's) | Seed Price (JojaMart) | Grow Time | Sell Price |
|-------------|------------------|------------------|-----------|------------|
| Blue Jazz   | 30               | 37               | 7         | 50         |
| Cauliflower | 80               | 100              | 12        | 175        |
| Green Bean  | 60               | 75               | 7 (3)    | 40         |
| Kale        | 70               | 87               | 6         | 110        |
| Parsnip     | 20               | 25               | 4         | 35         |
| Potato      | 50               | 62               | 6         | 100        |
| Strawberry  | 100              | -                | 8 (4)     | 120        |
| Tulip       | 20               | 25               | 6         | 30         |
| Rice        | 40               | -                | 6         | 30         |



Some crops (like Parsnips) are cheap to buy seeds for and grow quickly, but sell for less than more expensive crops, like Cauliflower. Green Beans and Strawberries can regrow, meaning that these plants will always grow more produce throughout the season (at the rate in parentheses) after the intial growing time.

In terms of seed cost, Pierre's is always the cheapest place to buy seeds, but on Wednesdays, his shop is closed. So the player must either forego buying seeds that day or buy them at a steeper price at the (implied Walmart equivalent) JojaMart. To complicate seed buying further, all stores are closed on two of the in-game days for festivals and Strawberries specifically can only be bought during the Egg Festival (Day 13) and during no other time.

## Aim
The goal is to optimise total gold earned by the end of the in-game month of Year 1 Spring (i.e. the beginning of the game). We are constraining the problem to Spring since this is the first month in the game, and optimising this time period could make future playthroughs somewhat easier.

The main constraints are as follows:
* The player starts with 500 gold on Day 1, and has until Day 28 to grow crops.
* The player also starts with 5 Parsnip seeds, which we assume they will start growing on Day 1.
* The gold can be exchanged for seeds of varying costs, usually depending on how much the resulting crop will sell for.
* The crop growing times increment each day, meaning that a parsnip that is planted on Day 1 and takes 5 days to grow will be ready to harvest on Day 6, not Day 5.
* Seeds whose grow times exceed the time available left in the season cannot be bought or grown. (This is possible in game, but results in "wasting" the seeds, and thus is suboptimal.)
* It is assumed the player will spend at all gold earned growing crops to buy more seeds for the remaining viable crops at the soonest possible opportunity.

## Model
To represent the maximization of profit by Day 28, the function can roughly be described as follows, where gold represents the variable we wish to maximize:

$$
\text{Gold at time t} = (\text{Produce sold by t}) - (\text{Seeds bought by t})
$$

The relationship between buying a crop and selling a crop at after $k$ days can be represented the following way, where $s_{Crop}(t)$ and $b_{Crop}(t)$ represents the number of crops of a specific type bought or sold at time $t$.:

$$
s_{Crop}(t+k) = -b_{Crop}(t)
$$

The number of seeds that can be bought at anytime is dependent upon the amount of gold available at one time, which itself is dependent upon the cash flow between days. Therefore, the algorithm must operate in the following manner:

* The algorithm starts at $t=1$ and $g(0)=500$, were $t$ and $g(t)$ correspond to time in (in-game) days and gold at time $t$.
* If there is gold to be spent and seeds that can be grown before Day 28 on time (Day) $t$, those seeds will be bought (usually at Pierre's) and planted on most days. If the current day is 3, 10, or 17, seeds wll be bought from Jojamart at a higher price instead (since Pierre's will be closed on those days). If the day is 13 or 24, the buy process is skipped entirely, as these are "Festival Days" in game where items cannot be bought or sold.
* If any crops mature by time $t$, they will be sold and the gold will be available the same day or (on days 3, 10, 13, 17, and 24) the following day. The constraint of the latter exists due to the shopkeeper not being available on the listed days.
* After all the daily processes take place, the day increases by 1 and the loop begins again until Day 29 is reached (meaning the end of the month has been reached).

## Optimisation Methods

The (theoretically) "best" solution would iterate through all combinations of seeds that could be bought on each day. However, this is not a practical solution because all future days are dependent on the buying decisions from the current day. Combine this with the fact that there are multiple seeds that can be bought almost every day, and this presents a challenge when figuring out how to optimise these decisions overtime.

### Method 1: Profit per Day Model

The first approach that was applied was one where, at every given opportunity, the algorithm chooses to always spend the most gold possible when it is available to buy the crops that yield the highest profit per day of growing. This is because the opportunity cost of buying seeds at one day means that there is less gold to use to buy seeds on all subsequent days (at least until the crop grows and is sold). This means that regrowables, which renew their crops after harvest, must have their profit per day recalculated depending on what day it is.

### Method 2: Profit per Day with Randomization

This approach is identical to the above approach except the rate at which all gold is used and the rate at which the algorithm chooses the highest "profit per day" crop varies. This is randomized because it is difficult to foresee cases when saving gold or buying "suboptimal" crops provides a better result than otherwise doing so.

### Methods 3 and 4: Quickest Turnaround (without and with Randomization)

Since time is a constraint, this method is similar to Method 1 and 2 except the seeds prioritized are the ones with the quickest grow rates. This is because while in the long run, profit per day may be the best method of generating gold, time is a limiting factor for our problem.

### Method 5: Highest "Interest"
This method prioritizes buying seeds based on the highest ROI adjusted for time. Specifically, the formula for this "interest" is as follows (for each crop):
$$
\text{Interest} = (\frac{\text{Seed Price}}{\text{Sell Price}})^{\text{Days to Harvest}}
$$

This is modified for the Green Bean and Strawberry crops in the following way, depending on the number of times these crops regenerate more crops after the initial harvest:
$$
\text{Interest} = (\frac{\text{Seed Price}}{\text{Sell Price}})^{\text{Days to Harvest}}+ (\frac{\text{Seed Price}}{\text{Sell Price}})^{\text{Days to Harvest}+\text{Days to Regrow}} + \dots
$$

### Method 6: Mix and Match

This method switches between the above methods in calculating the best seeds to buy during the "season" and uses only non-random methods to do this. There is likely a trade off in mid-term profitability between at least two of the above methods, so switching strategies at some point in the algorithm is likely a good choice.

In [3]:
# Import all csv data
seed_list = pd.read_csv('sd_opt_initial.csv')
joja_seed_list = pd.read_csv('sd_opt_joja.csv')
regrow = pd.read_csv('sd_opt_regrowth.csv')

# Remove crops not available in Year 1
seed_list = seed_list[seed_list['Year 2?'] == 0]
joja_seed_list = joja_seed_list[joja_seed_list['Year 2?'] == 0]

# Get lists of crops
crop_list = joja_seed_list[joja_seed_list['Seed Price'] < 1000]['Crop'].values.tolist()
regrow_list = regrow[joja_seed_list['Seed Price'] < 1000]['Crop'].values.tolist()

# Adjust Strawberry prices
egg_festival = seed_list.loc[8]
seed_list.loc[8,'Seed Price'] = 999
joja_seed_list.loc[8,'Seed Price'] = 999
egg_festival.loc['Profit per Day'] = 11.67

# Add profit per day to Joja data
joja_seed_list['Profit per Day'] = (joja_seed_list['Sell Price']-joja_seed_list['Seed Price'])/joja_seed_list['Days to Grow']

# Generate "interest" for crops
joja_seed_list['Interest'] = (joja_seed_list['Sell Price']/joja_seed_list['Seed Price']) ** (1/joja_seed_list['Days to Grow'])
seed_list['Interest'] = (seed_list['Sell Price']/seed_list['Seed Price']) ** (1/seed_list['Days to Grow'])

# Generate net profit for crops
joja_seed_list['Net Profit'] = joja_seed_list['Sell Price']-joja_seed_list['Seed Price']
seed_list['Net Profit'] = seed_list['Sell Price']-seed_list['Seed Price']

<ipython-input-3-11d3952f1e38>:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  regrow_list = regrow[joja_seed_list['Seed Price'] < 1000]['Crop'].values.tolist()
C:\Users\shoyr\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\shoyr\anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [4]:
# Display seed data (high seed prices are used in place of hard limits against selecting those crops)
seed_list

,Crop,Seed Price,Sell Price,Days to Grow,Year 2?,Profit per Day,Interest,Net Profit
0,Blue Jazz,37,50,7,0,1.857143,1.043954,13
1,Cauliflower,100,175,12,0,6.250000,1.047739,75
2,Kale,87,110,6,0,3.833333,1.039870,23
4,Green Bean,75,40,10,0,-3.500000,0.939074,-35
5,Parsnip,25,35,4,0,2.500000,1.087757,10
6,Potato,62,100,6,0,6.333333,1.082932,38
8,Strawberry,999,120,8,0,2.500000,0.767277,-879
9,Rice,9999,30,6,0,-1661.500000,0.379776,-9969
10,Tulip,25,30,6,0,0.833333,1.030853,5


In [5]:
# Initalize starting parameters
def init():
    # Starting gold
    gold = 500
    # First parsnips
    farm_plots = [['Parsnip', 5, 0]]
    # First day
    day = 1
    # Initialize dictionary of crop shopping list per day
    buy_order = {}
    return gold, day, farm_plots, buy_order

# Recalculates profit per day for green beans for optimising buying choices
def green_beans(day):
    if 29-day < 7:
        return 0
    else:
        i = day + 7
        amt = 40
        while i < 28:
            amt += 40
            i += 3
        return amt

# Recalculates "interest" for green beans based on the day
def green_beans_int(day, seed_list):
    if 29-day < 7:
        return 0
    else:
        i = day + 7
        amt = (seed_list.loc[4,'Sell Price']/seed_list.loc[4,'Seed Price'])**(1/(i-day))
        i += 3
        while i < 28:
            amt += (seed_list.loc[4,'Sell Price']/seed_list.loc[4,'Seed Price'])**(1/(i-day))
            i += 3
        return amt

# Sells crops to increase gold if harvest day has arrived
def sell_crops(gold, day, farm_plots):
    # Iterate through all "plots" on farm
    i = 0
    while i < len(farm_plots):
        crop_name = farm_plots[i][0]
        crop_amt = farm_plots[i][1]
        crop_day = farm_plots[i][2]
        
        # If crop is of harvest age, either harvest and remove or regrow if the crop is regrowable
        if crop_day >= int(seed_list[seed_list['Crop'] == crop_name]['Days to Grow']):
            if crop_name in regrow_list:
                farm_plots[i] = [crop_name,crop_amt,0,'r']
                gold += int(seed_list[seed_list['Crop'] == crop_name]['Sell Price'] * crop_amt)
                i += 1
            else:
                gold += int(seed_list[seed_list['Crop'] == crop_name]['Sell Price'] * crop_amt)
                del farm_plots[i]
        # Else if the crop is a regrowable, is tagged with 'r', and of regrow harvest age, harvest and reset age
        elif crop_name in regrow_list:
            if crop_day >= int(regrow[regrow['Crop'] == crop_name]['Days to Grow']) and len(farm_plots[i]) > 3:
                farm_plots[i] = [crop_name,crop_amt,0,'r']
                gold += int(seed_list[seed_list['Crop'] == crop_name]['Sell Price'] * crop_amt)
            i += 1
        # But if the crop still needs to grow, skip to next item
        else:
            i += 1

    return gold, day, farm_plots

# Chooses crops available based on day and available gold
def cycle_buyable_crops(gold,day,limit=28):
    buyables = pd.Series(data=None, dtype=object)
    if day not in [3,10,13,17,24]:
        buyables = seed_list[(seed_list['Seed Price'] <= gold)]
        buyables = buyables[buyables['Days to Grow'] <= limit-day]
    elif day not in [13, 24]:
        buyables = joja_seed_list[(joja_seed_list['Seed Price'] <= gold)]
        buyables = buyables[buyables['Days to Grow'] <= limit-day]
    elif day == 13 and gold >= 100:
        buyables = egg_festival
    return buyables

# Add a given crop to the buy order dictionary for that day (or increment amount if already present)
def append_to_record(buy_order, crop_name, crop_amt,day):
    if buy_order.get(day):
        if buy_order[day].get(crop_name):
            buy_order[day][crop_name] += crop_amt
        else:
            buy_order[day][crop_name] = crop_amt
    else:
        buy_order[day] = {crop_name: crop_amt}
    return buy_order

# Cycle through buyable crop list to buy crop, usually of highest profit per day
def buy_crops(gold, day, farm_plots, buy_order,diversion_rate=0.05,selection='ppd',limit=28):
    # Initalize buyable crop list
    buyables = cycle_buyable_crops(gold,day,limit)
    
    # Initalize size of farm plot list (to break while loop if no purchase can be made)
    initial_length = len(farm_plots)
              
    # Buy either the best profit per day crop or select randomly based on diversion_rate
    # Note: if it's the 13th day, no other choice exists except for the Strawberry crop
    while len(buyables) > 0:
        if random.uniform(0,1) > diversion_rate:
            if day == 13:
                crop_name,crop_amt,crop_day = buyables['Crop'], 1, 0
                gold -= buyables['Seed Price']
                farm_plots.append([crop_name,crop_amt,crop_day])
                buy_order = append_to_record(buy_order, crop_name, crop_amt, day)
                buyables = cycle_buyable_crops(gold,day,limit)
            elif random.uniform(0,1) > diversion_rate:
                # Choose method of choosing crop at 1-diversion_rate
                if selection == 'ppd':
                    buy_index = int(buyables[['Profit per Day']].idxmax())
                elif selection == 'turnover':
                    buy_index = int(buyables[['Days to Grow']].idxmin())
                elif selection == 'interest':
                    buy_index = int(buyables[['Interest']].idxmax())
                elif selection == 'tp':
                    buy_index = int(buyables[['Net Profit']].idxmax())
                else:
                    buy_index = int(buyables[['Profit per Day']].idxmax())
                crop_name,crop_amt,crop_day = buyables.loc[buy_index]['Crop'], 1, 0
                gold -= buyables.loc[buy_index]['Seed Price']
                farm_plots.append([crop_name,crop_amt,crop_day])
                buy_order = append_to_record(buy_order, crop_name, crop_amt, day)
                buyables = cycle_buyable_crops(gold,day,limit)
            else:
                # Choose crop randomly from available choices
                buy_index = random.choice(buyables.index.tolist())
                crop_name,crop_amt,crop_day = buyables.loc[buy_index]['Crop'], 1, 0
                gold -= buyables.loc[buy_index]['Seed Price']
                farm_plots.append([crop_name,crop_amt,crop_day])
                buy_order = append_to_record(buy_order, crop_name, crop_amt, day)
                buyables = cycle_buyable_crops(gold,day,limit)
        else:
            break
    return gold, day, farm_plots, buy_order

In [6]:
# Cycles through buy/sell processes and updates green bean prices; returns results with day incremented
def tick_over(gold,day,farm_plots,buy_order,diversion_rate=0.05,selection='ppd',limit=28):
    # Green bean profitability mod
    g = green_beans(day)
    joja_seed_list.loc[4,'Profit per Day'] = (g-joja_seed_list.loc[4,'Seed Price']) / (limit+1-day)
    seed_list.loc[4,'Profit per Day'] = (g-seed_list.loc[4,'Seed Price']) / (limit+1-day)
    #Interest
    joja_seed_list.loc[4,'Interest'] = green_beans_int(day,joja_seed_list)
    seed_list.loc[4,'Interest'] = green_beans_int(day,seed_list)
    
    # Selling and buying processes
    gold,day,farm_plots = sell_crops(gold,day,farm_plots)
    gold,day,farm_plots,buy_order = buy_crops(gold,day,farm_plots,buy_order,diversion_rate,selection,limit)
    
    # Increase age of all plants
    for i in range(len(farm_plots)):
        farm_plots[i][2] += 1
    return gold, day+1, farm_plots,buy_order

In [7]:
# Combine all above functions to iterate over days and return all main parameters
def generate_run(days=28,diversion_rate=0.05,selection='ppd',starting_params=()):
    if not starting_params:
        gold, day, farm_plots, buy_order = init()
        starting_params = (gold, day, farm_plots, buy_order)
    else:
        gold, day, farm_plots, buy_order = starting_params
    for i in range(days-starting_params[1]):
        gold, day, farm_plots, buy_order = tick_over(gold, day, farm_plots, buy_order,diversion_rate,selection,limit=28)
    return gold, day+1, farm_plots, buy_order

## Results
### Method 1
Using Method 1, the purchase order for crop seeds is as follows:
* Day 1: 6 Green Beans, 2 Parsnips
* Day 5: 5 Green Beans, 1 Parsnip
* Day 9: 2 Parsnips
* Day 11: 3 Green Beans
* Day 14: 5 Potatoes
* Day 15: 3 Potatoes, 1 Parsnip
* Day 17: 4 Potatoes
* Day 18: 1 Potato, 2 Parsnips
* Day 19: 1 Parsnip
* Day 20: 10 Potatoes, 1 Parsnip
* Day 21: 7 Potatoes
* Day 22: 1 Potato, 1 Parsnip
* Day 23: 27 Parsnips
By the end of Day 28, this results in the player having 3537 gold.

In [8]:
baseline_run = generate_run(days=28,diversion_rate=0)
print(baseline_run[0],baseline_run[3])

3537 {1: {'Green Bean': 6, 'Parsnip': 2}, 5: {'Green Bean': 3}, 11: {'Green Bean': 3, 'Parsnip': 1}, 14: {'Potato': 4}, 15: {'Potato': 2, 'Parsnip': 1}, 17: {'Potato': 4}, 18: {'Potato': 1, 'Parsnip': 2}, 19: {'Parsnip': 1}, 20: {'Potato': 10, 'Parsnip': 1}, 21: {'Potato': 7}, 22: {'Potato': 1, 'Parsnip': 1}, 23: {'Parsnip': 27}}


### Method 2

Modifying Method 1 to include some randomness and running the algorithm en mass provides somewhat better results. However, this improvement comes at the cost of a much higher runtime. The absolute best path is the following (from using the diversion rate 0.1):

* Day 1: 6 Green Beans, 2 Parsnips
* Day 5: 3 Green Beans
* Day 12: 4 Potatoes
* Day 14: 4 Potatoes
* Day 15: 2 Potatoes
* Day 17: 3 Potatoes, 2 Parsnips
* Day 18: 8 Potatoes, 1 Parsnip
* Day 20: 10 Potatoes
* Day 21: 6 Potatoes, 1 Parsnip
* Day 22: 2 Parsnips
* Day 23: 21 Parsnips

This order of buying seeds yields 3736 gold by the end of Day 28 (an improvement of 199 gold over the non-randomized algorithm).

In [9]:
div5 = max([generate_run(diversion_rate=0.05) for i in range(200)], key=lambda x: x[0])
div10 = max([generate_run(diversion_rate=0.1) for i in range(200)], key=lambda x: x[0])

print('Max Profit (Diversion rate 0.05)')
print('Gold: {}'.format(div5[0]))
print('Purchase Order: {}'.format(div5[-1]))

print('Max Profit (Diversion rate 0.10)')
print('Gold: {}'.format(div10[0]))
print('Purchase Order: {}'.format(div10[-1]))

Max Profit (Diversion rate 0.05)
Gold: 3686
Purchase Order: {1: {'Green Bean': 4}, 2: {'Blue Jazz': 1, 'Green Bean': 2}, 5: {'Green Bean': 2, 'Parsnip': 1}, 9: {'Potato': 1, 'Parsnip': 1}, 11: {'Parsnip': 1, 'Green Bean': 1, 'Potato': 1}, 12: {'Potato': 1, 'Parsnip': 1}, 14: {'Potato': 3}, 15: {'Potato': 4, 'Parsnip': 2}, 16: {'Parsnip': 1}, 17: {'Potato': 4, 'Parsnip': 1}, 19: {'Potato': 5, 'Parsnip': 1}, 20: {'Potato': 4}, 21: {'Potato': 13, 'Tulip': 1, 'Parsnip': 2}, 23: {'Parsnip': 23}}
Max Profit (Diversion rate 0.10)
Gold: 3586
Purchase Order: {1: {'Green Bean': 6, 'Parsnip': 2}, 5: {'Green Bean': 3}, 12: {'Potato': 4}, 14: {'Potato': 4}, 15: {'Potato': 2}, 17: {'Potato': 3, 'Parsnip': 2}, 18: {'Potato': 8, 'Parsnip': 1}, 20: {'Potato': 8, 'Kale': 1, 'Blue Jazz': 1}, 21: {'Potato': 6, 'Parsnip': 1}, 23: {'Parsnip': 12}}


## Method 3
The general buying function is the same, except instead of prioritizing profit per day, the function now looks for the crop with the lowest time to harvest (which means the algorithm will always buy Parsnips except for on the 13th).

In [10]:
baseline_run = generate_run(days=28,diversion_rate=0,selection='turnover')
print(baseline_run[0],baseline_run[3])

4915 {1: {'Parsnip': 20}, 5: {'Parsnip': 35}, 9: {'Parsnip': 49}, 13: {'Strawberry': 17}, 21: {'Parsnip': 82}}


The result is fairly boring for Method 3. Essentially, it amounts to buying Parsnips (or Strawberries) at every possible opportunity to do so, and the player will end up with 4915 gold by the end of the month.

## Method 4
This method involves adding some randomness into the Method 3 buying decisons. However, since the point of these last two Methods is to optimize crop output (in hopes it will result in greater profits in the short term), the buy function has been modified so that the algorithm will always use up gold to buy growable seeds until not enough gold remains to buy more.

The best buy order below generated at a diversion rate of 0.05 yields 4420 gold and replaces some of the Parsnip crops for the occasional other crop. However, it does not outperform Method 3.

In [11]:
div5 = max([generate_run(diversion_rate=0.05,selection='turnover') for i in range(200)], key=lambda x: x[0])
div10 = max([generate_run(diversion_rate=0.1,selection='turnover') for i in range(200)], key=lambda x: x[0])

print('Max Profit (Diversion rate 0.05)')
print('Gold: {}'.format(div5[0]))
print('Purchase Order: {}'.format(div5[-1]))

print('Max Profit (Diversion rate 0.10)')
print('Gold: {}'.format(div10[0]))
print('Purchase Order: {}'.format(div10[-1]))

Max Profit (Diversion rate 0.05)
Gold: 4346
Purchase Order: {1: {'Parsnip': 17}, 2: {'Parsnip': 1}, 3: {'Parsnip': 2}, 5: {'Parsnip': 25, 'Cauliflower': 1, 'Tulip': 1}, 6: {'Parsnip': 2}, 7: {'Parsnip': 3}, 9: {'Parsnip': 35}, 10: {'Tulip': 1, 'Parsnip': 1}, 11: {'Parsnip': 6}, 13: {'Strawberry': 12}, 14: {'Parsnip': 2}, 15: {'Parsnip': 8, 'Tulip': 1}, 16: {'Parsnip': 1}, 17: {'Parsnip': 7}, 18: {'Parsnip': 3}, 19: {'Parsnip': 11}, 20: {'Parsnip': 1}, 21: {'Parsnip': 38, 'Tulip': 1, 'Kale': 1}, 22: {'Parsnip': 14, 'Kale': 1}, 23: {'Parsnip': 19}}
Max Profit (Diversion rate 0.10)
Gold: 3616
Purchase Order: {1: {'Parsnip': 5}, 2: {'Parsnip': 12, 'Green Bean': 1}, 5: {'Parsnip': 11, 'Green Bean': 1}, 6: {'Parsnip': 6}, 7: {'Parsnip': 2}, 8: {'Parsnip': 8}, 9: {'Parsnip': 14}, 10: {'Parsnip': 4}, 11: {'Parsnip': 3}, 12: {'Parsnip': 2}, 13: {'Strawberry': 9}, 14: {'Parsnip': 2, 'Cauliflower': 1}, 15: {'Parsnip': 5, 'Blue Jazz': 1, 'Tulip': 1}, 16: {'Parsnip': 2}, 18: {'Parsnip': 1}, 19: {'P

## Method 5

Choosing crops based on delayed "interest" greatly underperforms compared to other methods. This method prefers buying green beans early, but it appears that the profit from this approach does not increase quickly enough within the 28 day timescale to outperform other methods. The optimal route for this method only yields 2505 gold, nearly half of the previous method.

In [12]:
baseline_run = generate_run(days=28,diversion_rate=0,selection='interest')
print(baseline_run[0],baseline_run[3])

2505 {1: {'Green Bean': 6, 'Parsnip': 2}, 5: {'Green Bean': 3}, 11: {'Green Bean': 3, 'Parsnip': 1}, 14: {'Green Bean': 3, 'Parsnip': 1}, 15: {'Green Bean': 2}, 17: {'Green Bean': 3}, 18: {'Parsnip': 7}, 20: {'Parsnip': 9}, 21: {'Parsnip': 10}, 22: {'Parsnip': 10}, 23: {'Parsnip': 9}}


## Method 6
The result of combining methods of choosing selections does not improve beyond the results from Method 3. The code below only "changes" methods once (though the best result never changes methods at all), but the same loop was run with three "changes" in selections, but this was done in a separate notebook, so the results were not saved here. However, the result was the same buy order from the algorithm that prioritizes crop turnover (i.e. all Parsnips)

In [13]:
# Try all permutations
np.seterr(divide='ignore')

runs = []
selections = ['ppd','turnover','interest','tp']
for i in range(5,28):
    for j in selections:
        for k in selections:
            r1 = generate_run(days=i, diversion_rate=0, selection=j)
            r2 = generate_run(days=28, diversion_rate=0, selection=k, starting_params=r1)
            runs.append(r2 + (j,k,i))
best = max(runs, key=lambda x: x[0])
print(best[0],best[3],best[4:])

4915 {1: {'Parsnip': 20}, 5: {'Parsnip': 35}, 9: {'Parsnip': 49}, 13: {'Strawberry': 17}, 22: {'Parsnip': 82}} ('turnover', 'turnover', 14)


In [ ]:
# This did not produce a different buy order compared to the above
np.seterr(divide='ignore')

runs = []
selections = ['ppd','turnover','interest','tp']
for i in range(5,28):
    for l in range(i,28):
        for j in selections:
            for k in selections:
                for m in selections:
                    r1 = generate_run(days=i, diversion_rate=0, selection=j)
                    r2 = generate_run(days=l, diversion_rate=0, selection=k, starting_params=r1)
                    r3 = generate_run(days=28, diversion_rate=0, selection=m, starting_params=r2)
                    runs.append(r3 + (j,k,i))
best = max(runs, key=lambda x: x[0])
print(best[0],best[3],best[4:])

## Conclusion
The results from our algorithm suggest that crop turnover is the most important thing in terms of optimizing the gold that is received by the end of the season. It appears that the overall profitability of crops depends most on the ability to sell them quickly after planting them instead of net profit. As can be seen below, even when the time scale for the season is inflated to be longer than it actually is in game, crops with high turnover consistently outperform all other selection methods, with flat net profit being the worst valuation method. These methods yield 70,225 and 19,282 gold respectively.

In [ ]:
np.seterr(divide='ignore')

# Generate runs over 50 days instead of 28
def generate_run(days=28,diversion_rate=0.05,selection='ppd',starting_params=()):
    if not starting_params:
        gold, day, farm_plots, buy_order = init()
        starting_params = (gold, day, farm_plots, buy_order)
    else:
        gold, day, farm_plots, buy_order = starting_params
    for i in range(days-starting_params[1]):
        gold, day, farm_plots, buy_order = tick_over(gold, day, farm_plots, buy_order,diversion_rate,selection,limit=50)
    return gold, day+1, farm_plots, buy_order

selections = ['ppd','turnover','interest','tp']
x = [(generate_run(days=50, diversion_rate=0, selection=m), m) for m in selections]
best_long = max(x, key=lambda x: x[0][0])
print(best_long[0][0],best_long[0][-1],best_long[1])

In [ ]:
# Print gold output for each selection strategy
for i in x:
    print(i[0][0],i[1])

## Model Critique

The model is useful in that it illustrates how powerful crops with low growing times are if the goal is to grow the most profitable crops. While it is hard to not make money farming in the game, it is difficult to figure out how to optimise seed buying decisions to maximize profitability. (At least after planting enough of the other crops to get certain in-game rewards). Most guides for Stardew Valley will list "Profit per Day" when listing out crops for each season. However, the results of the model clearly that the opportunity cost of planting crops that take a lot of time to grow compared to ones that can be harvested quicker is significant and should actually be taken into account to anyone who would like to optimise their playthrough. (However, brutal optimisation this is actually somewhat antithetical to the ethos the game attempts to portray, but this is beside the point.)

The model might be improved further if it could evaluate the tradeoff between saving gold on certain days to buy Strawberry seeds on day 13. This could not be accounted for in the algorithmn as it currently exists since it assumes that the player will always use gold at the earliest opportunities. The *diversion_rate* component of this algorithm may also be improved if the algorithm was limited to choosing the second best choices according to their selection criteria rather than all available seeds.

The model also does not account for the following game mechanics that would influence the seed buying decisions overtime:
* **Fatigue**: The player must use energy to plow fields to plant the seeds, then use more energy to water the seeds and crops during the growing time. This means that realistically, fatigue can limit the player's ability to keep crops growing overtime. This would most effect crops like Parsnips, which the current model suggests the player should buy in high numbers. Conversely, crops like Cauliflower, which are more expensive to buy but payout more per seed, may be favored more in a model that factors in the upper limits of fatigue. Furthermore, energy use for plowing and watering decreases as the player harvests more crops, which would also influence this theoretical model.
* **Other Income and Expenses**: The player can also earn gold from other methods, such as fishing, mining, and foraging. The model does not account for the income from these activities nor does it account for the player buying items either relating to these activities among other reasons. The model could be written to be more complex either to allow gold to be decremented from the farm (such as to buy certain items) or to account for income from other sources that could then be used to buy more seeds.
* **Crop Quality**: The crop sell prices actually do vary considerably depending on the "quality" of the crop, which are rated as such with a silver, gold, and iridium (made up rare element) stars if it is above the base quality. The chances of getting a higher quality version of the crop from a given harvest depends on either using certain items on the plowed land before planting the seeds and by how high the farmer's "farming level" is. Since the current model assumes no item use and assumes all crops produced are of regular quality, the results of the model actually understate the amount of gold that will be earned on average per crop and actually better approximates the lower limit of what is possible.

# Works Cited
[1] Concerned Ape, Seattle, WA, USA, *Stardew Valley*, 2016 [Steam Version]